In [5]:
import mammoth
from IPython.core.display import display, HTML
import subprocess
import os
import glob
import string
import stat
import random
from bs4 import BeautifulSoup
import re
from difflib import SequenceMatcher
import shutil 
import filecmp
import difflib
import pickle

In [ ]:
# COUNT THE DOC AND DOCX FILES

allDocFilesGlob = "/Users/mettinger/Desktop/2018_AS/**/*.docx"
allFilesDocx = glob.glob(allDocFilesGlob, recursive=True)

allDocFilesGlob = "/Users/mettinger/Desktop/2018_AS/**/*.doc"
allFilesDoc = glob.glob(allDocFilesGlob, recursive=True)

print("doc: %s\ndocx: %s" % (str(len(allFilesDoc)), str(len(allFilesDocx))))

In [ ]:
# CONVERT FROM DOC TO DOCX SO MAMMOTH CAN CONVERT TO HTML

docFilesConverted = []
for i,filename in enumerate(allFilesDoc):
    if i % 100 == 0:
        print(i)
    os.chdir('/'.join(filename.split('/')[0:-1]))
    if filename.endswith('.doc'):
        docFilesConverted.append(filename)
        subprocess.call(['/Applications/LibreOffice.app/Contents/MacOS/soffice', '--headless', '--convert-to', 'docx', filename])

In [ ]:
# COUNT THE DOC AND DOCX FILES

allDocFilesGlob = "/Users/mettinger/Desktop/2018_AS/**/*.docx"
allFilesDocx = glob.glob(allDocFilesGlob, recursive=True)

allDocFilesGlob = "/Users/mettinger/Desktop/2018_AS/**/*.doc"
allFilesDoc = glob.glob(allDocFilesGlob, recursive=True)

print("doc: %s\ndocx: %s" % (str(len(allFilesDoc)), str(len(allFilesDocx))))

In [ ]:
allDocFilesGlob = "/Users/mettinger/Desktop/2018_AS/**/*.docx"
allFilesDocx = glob.glob(allDocFilesGlob, recursive=True)

numFile = len(allFilesDocx)

for i in range(10):
    os.mkdir("/Users/mettinger/Desktop/rockArtProjectData/anikaDocx" + str(i))
    
for i,filePath in enumerate(allFilesDocx):
    fileName = filePath.split("/")[-1]
    index = str(int( (i * 10) / numFile))
    destPath = "/Users/mettinger/Desktop/rockArtProjectData/anikaDocx" + index + "/" + fileName
    shutil.copyfile(filePath, destPath)


In [ ]:
# REMOVE TILDA FILES

allTildaFilesGlob = "/Users/mettinger/Desktop/rockArtProjectData/**/~*.docx"
allTildaFiles = glob.glob(allTildaFilesGlob, recursive=True)

for thisFile in allTildaFiles:
    os.remove(thisFile)
print("tilda removed: " + str(len(allTildaFiles)))

In [ ]:
# FIND IDENTICAL FILES

allDocFilesGlob = "/Users/mettinger/Desktop/rockArtProjectData/**/*.docx"
allFilesDocx = glob.glob(allDocFilesGlob, recursive=True)
print(len(allFilesDocx))

identicalDict = {}

for i,filename in enumerate(allFilesDocx):
    flag = False
    if i % 1000 == 0:
        print(i)
    for thisKey in identicalDict.keys():
        if filecmp.cmp(thisKey, filename):
            identicalDict[thisKey].append(filename)
            flag = True
            break
    if not flag:
        identicalDict[filename] = []
        
print("repeated files: " + str(sum([len(value) for key,value in identicalDict.items()])))

In [ ]:
# DELETE IDENTICAL FILES

for thisKey in identicalDict.keys():
    thisFileList = identicalDict[thisKey]
    for thisFile in thisFileList:
        os.remove(thisFile)

In [ ]:
# CHECK IDENTICAL DOCX FILES HAVE BEEN ELIMINATED

allDocFilesGlob = "/Users/mettinger/Desktop/rockArtProjectData/**/*.docx"
allFilesDocx = glob.glob(allDocFilesGlob, recursive=True)
print(len(allFilesDocx))

identicalDict = {}

for i,filename in enumerate(allFilesDocx):
    flag = False
    if i % 1000 == 0:
        print(i)
    for thisKey in identicalDict.keys():
        if filecmp.cmp(thisKey, filename):
            identicalDict[thisKey].append(filename)
            flag = True
            break
    if not flag:
        identicalDict[filename] = []
        
print("repeated files: " + str(sum([len(value) for key,value in identicalDict.items()])))

In [ ]:
def stripInfo(soup):
    regExpList = [r'Coffman', r'149 Atlantic',r'Swampscott',r'\$\d*\.\d\d']
    for thisRegExp in regExpList:
        for elem in soup(text=re.compile(thisRegExp)):
            elem.parent.decompose()

In [ ]:
# CONVERT TO HTML, DELETE SENSITIVE INFORMATION (ADDRESS, ORDER INFO, ETC....), SAVE HTML FILE

allDocFilesGlob = "/Users/mettinger/Desktop/rockArtProjectData/**/*.docx"
allFiles = glob.glob(allDocFilesGlob, recursive=True)
convertErrors = []
regExErrors = []
writeErrors = []
print(len(allFiles))

for i,filename in enumerate(allFiles):
    if i % 1000 == 0:
        print(i)
    with open(filename, "rb") as docx_file:
        try:
            result = mammoth.convert_to_html(docx_file)
            html = result.value
            soup = BeautifulSoup(html, 'html.parser')
            try:
                stripInfo(soup)
                html = str(soup)
                try:
                    thisFileName = filename[0:-5] + ".html"
                    with open(thisFileName, "w") as thisHtmlFile:
                        thisHtmlFile.write(str(html))
                except:
                    writeErrors.append(filename)
            except:
                regExErrors.append(filename)
        except:
            convertErrors.append(filename)
    
len(writeErrors), len(regExErrors), len(convertErrors)

In [ ]:
# FIND IDENTICAL HTML FILES

allFilesGlob = "/Users/mettinger/Desktop/rockArtProjectData/**/*.html"
allFiles = glob.glob(allFilesGlob, recursive=True)
print(len(allFiles))

identicalDict = {}

for i,filename in enumerate(allFiles):
    flag = False
    if i % 1000 == 0:
        print(i)
    for thisKey in identicalDict.keys():
        if filecmp.cmp(thisKey, filename):
            identicalDict[thisKey].append(filename)
            flag = True
            break
    if not flag:
        identicalDict[filename] = []
        
print("repeated files: " + str(sum([len(value) for key,value in identicalDict.items()])))

In [ ]:
# DELETE IDENTICAL HTML FILES AND CORRESPONDING DOCX FILES

htmlRemoveErrors = []
docxRemoveErrors = []

for thisKey in identicalDict.keys():
    thisFileList = identicalDict[thisKey]
    for thisFile in thisFileList:
        thisDocxFile = thisFile[0:-5] + ".docx"
        
        try:
            os.remove(thisFile)
        except:
            htmlRemoveErrors.append(thisFile)
        try:
            os.remove(thisDocxFile)
        except:
            docxRemoveErrors.append(thisDocxFile)
            
len(htmlRemoveErrors), len(docxRemoveErrors)            

In [ ]:
# CHECK IDENTICAL HTML FILES HAVE BEEN ELIMINATED

allFilesGlob = "/Users/mettinger/Desktop/rockArtProjectData/**/*.html"
allFiles = glob.glob(allFilesGlob, recursive=True)
print(len(allFiles))

identicalDict = {}

for i,filename in enumerate(allFiles):
    flag = False
    if i % 1000 == 0:
        print(i)
    for thisKey in identicalDict.keys():
        if filecmp.cmp(thisKey, filename):
            identicalDict[thisKey].append(filename)
            flag = True
            break
    if not flag:
        identicalDict[filename] = []
        
print("repeated files: " + str(sum([len(value) for key,value in identicalDict.items()])))

In [ ]:
# MEASURE TO DETECT DUPLICATES AND NEAR DUPLICATES

allFilesGlob = "/Users/mettinger/Desktop/rockArtProjectData/**/*.html"
allFiles = glob.glob(allFilesGlob, recursive=True)
print(len(allFiles))

simHTMLList = []

for i,firstPath in enumerate(allFiles):
    if i % 1000 == 0:
        print(i)
    for j in range(i+1, len(allFiles)):
        secondPath = allFiles[j]
        score = SequenceMatcher(None, firstPath, secondPath).ratio()
        if score > .9:
            simHTMLList.append((firstPath, secondPath, score))
        
simHTMLList = sorted(simHTMLList, key = lambda x: x[2], reverse = True)


In [ ]:
simHTMLList[0:1000]

In [ ]:
htmlFile1, htmlFile2, simScore = simHTMLList[20]

with open(htmlFile1, 'r') as fp:
    html1 = fp.readlines()[0]
    
with open(htmlFile2, 'r') as fp:
    html2 = fp.readlines()[0]

print(htmlFile1)
print(htmlFile2)
print(simScore)

In [ ]:
display(HTML(html1))

In [ ]:
display(HTML(html2))

In [ ]:
# MEASURE SIMILARITY OF ALL FILENAMES TO DETECT DUPLICATES AND NEAR DUPLICATES

allFilesGlob = "/Users/mettinger/Desktop/rockArtProjectData/**/*.html"
allFiles = glob.glob(allFilesGlob, recursive=True)
print(len(allFiles))

simNameList = []

for i in range(len(allFiles)):
    if i % 1000 == 0:
        print(i)
    firstPath = allFiles[i]
    firstName = firstPath.split('/')[-1].split('.')[0]
    for j in range(i + 1, len(allFiles)):
        secondPath = allFiles[j]
        secondName = secondPath.split('/')[-1].split('.')[0]
        score = SequenceMatcher(None, firstName, secondName).ratio()
        simNameList.append((firstPath, secondPath, score))
        
simNameList = sorted(simNameList, key = lambda x: x[2], reverse = True)

print("identical filenames: " + str(len([i for i in simList if i[2] == 1])))

In [ ]:
# WRITE SIMILARITY FILE

numEntries = 1000

with open('/Users/mettinger/Desktop/similarList.txt', 'w') as f:
    for item in simList[0:numEntries]:
        f.write("%s\n" % item[0])
        f.write("%s\n" % item[1])
        f.write("%s\n" % str(item[2]))
        f.write("\n\n")

In [7]:
# CREATE MAPPING FROM ALL ANNIKA FILES BACK TO 2018_AS DIRECTORY STRUCTURE

annikaDictPicklePath = '/Users/mettinger/Data/rockArtProject/annikaDict.pkl'

try:
    annikaDict = pickle.load(annikaDictPicklePath)
except:
    allDocFilesGlob = "/Users/mettinger/Desktop/2018_AS/**/*.docx"
    allFilesDocx = glob.glob(allDocFilesGlob, recursive=True)

    numFile = len(allFilesDocx)
    annikaFileList = []

    for i,filePath in enumerate(allFilesDocx):
        fileName = filePath.split("/")[-1]
        index = str(int( (i * 10) / numFile))
        destPath = "/Users/mettinger/Desktop/rockArt/original/anikaDocx" + index + "/" + fileName
        annikaFileList.append((destPath, filePath))

    annikaDict = dict(annikaFileList)
    pickle.dump( annikaDict, open( annikaDictPicklePath, "wb" ) )

In [4]:
# CHECK FOR BAD FILES WITHOUT ANNIKA MAPPING

allCurrentGlob = "/Users/mettinger/Desktop/rockArt/original/**/*.docx"
allFiles = glob.glob(allCurrentGlob, recursive=True)

badFiles = []
for filePath in allFiles:
    if filePath not in annikaDict:
        badFiles.append(filePath)

badFiles

['/Users/mettinger/Desktop/rockArt/original/anikaDocx3/A179-Asia-Indus-Harappa-Bajaur Valley-Priestess-2800-2600 BCE.docx',
 '/Users/mettinger/Desktop/rockArt/original/anikaDocx8/~$00-AM,S-Peru-Moche-Jaguar Shaman on Gooseback-Ceramic-100-700 CE.docx',
 '/Users/mettinger/Desktop/rockArt/original/anikaDocx6/~$00-Afr-Mali-Dogon-Nyérum-Female Water Spirit on a Stool.docx',
 '/Users/mettinger/Desktop/rockArt/original/anikaDocx6/Afr-Sierra Leone-Nommo-Ruetimeyer-1901 Final.docx']

In [15]:
# RECREATE 2018_AS structure

allCurrentGlob = "/Users/mettinger/Desktop/rockArt/original/**/*.docx"
allFiles = glob.glob(allCurrentGlob, recursive=True)

for thisFile in allFiles:
    if thisFile in annikaDict:
        destination = annikaDict[thisFile]
        destination = destination.replace('2018_AS','2018_AS_Processed')
        if not os.path.exists(os.path.dirname(destination)):
            os.makedirs(os.path.dirname(destination))
        shutil.copyfile(thisFile, destination)
    else:
        print("Annika dict error: " + thisFile)

Annika dict error: /Users/mettinger/Desktop/rockArt/original/anikaDocx3/A179-Asia-Indus-Harappa-Bajaur Valley-Priestess-2800-2600 BCE.docx
Annika dict error: /Users/mettinger/Desktop/rockArt/original/anikaDocx8/~$00-AM,S-Peru-Moche-Jaguar Shaman on Gooseback-Ceramic-100-700 CE.docx
Annika dict error: /Users/mettinger/Desktop/rockArt/original/anikaDocx6/~$00-Afr-Mali-Dogon-Nyérum-Female Water Spirit on a Stool.docx
Annika dict error: /Users/mettinger/Desktop/rockArt/original/anikaDocx6/Afr-Sierra Leone-Nommo-Ruetimeyer-1901 Final.docx
